_**NOTE: Unfinished notebook. The kernel got suspsended during similarity calculation**_

_TODO: Fix bug_

In [1]:
import os
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [3]:
# load model
model_dir = "./models"
model_h5_file = os.path.join(model_dir, "model_task_2_cnn_classification.h5")

model = load_model(model_h5_file)
model

In [ ]:
bins = 88

def get_histogram(image: np.ndarray):
    histogram = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    histogram = cv2.normalize(histogram, histogram).flatten()
    return histogram 

def get_texture_feature(image: np.ndarray):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = texture.haralick(gray_image)
    return np.mean(glcm, axis=0)

def get_compactness(image: np.ndarray):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive thresholding
    thresholded = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate compactness for each contour
    compactness_values = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        compactness = perimeter / np.sqrt(area) if area > 0 else 0
        compactness_values.append(compactness)
    
    # Maximum Compactness: Choose the contour with the highest compactness value. 
    # This approach assumes that the object with the highest compactness is the most significant or relevant in the image.
    max_compactness_index = np.argmax(compactness_values)
    max_compactness_value = compactness_values[max_compactness_index]
    
    return max_compactness_value


def get_similarity_attrs(row: pd.Series) -> pd.Series: 
    print(row['path'])
    image = cv2.imread(row['path'])

    row['histogram'] = get_histogram(image)
    row['texture_feature'] = get_texture_feature(image)
    row['compactness'] = get_compactness(image)

    return row 

df_similarity = df.progress_apply(get_similarity_attrs, axis=1)